<a href="https://colab.research.google.com/github/ravi-gopalan/ashrae/blob/master/ashrae_energy_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd /root

/root


In [0]:
!mkdir .kaggle 

In [0]:
!cp '/gdrive/My Drive/Colab_stuff/kaggle.json' '/root/.kaggle/kaggle.json' 
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
cd /content/

/content


In [0]:
!mkdir ashrae_energy_prediction

In [7]:
cd ashrae_energy_prediction/

/content/ashrae_energy_prediction


In [8]:
!kaggle competitions download -c ashrae-energy-prediction

 82% 137M/167M [00:01<00:00, 63.7MB/s]
100% 167M/167M [00:01<00:00, 119MB/s] 
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 184MB/s]
 96% 85.0M/88.4M [00:00<00:00, 85.1MB/s]
100% 88.4M/88.4M [00:00<00:00, 114MB/s] 
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 172MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 39.9MB/s]
 88% 105M/120M [00:01<00:00, 80.2MB/s] 
100% 120M/120M [00:01<00:00, 99.1MB/s]


In [9]:
!unzip train.csv.zip
!unzip test.csv.zip
!unzip weather_train.csv.zip
!unzip weather_test.csv.zip
!unzip sample_submission.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  weather_train.csv.zip
  inflating: weather_train.csv       
Archive:  weather_test.csv.zip
  inflating: weather_test.csv        
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [10]:
!ls

building_metadata.csv	   test.csv.zip      weather_test.csv.zip
sample_submission.csv	   train.csv	     weather_train.csv
sample_submission.csv.zip  train.csv.zip     weather_train.csv.zip
test.csv		   weather_test.csv


# Library Imports

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
import gc
import sys

import lightgbm as lgb

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Helper functions

In [0]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Building MetaData EDA

In [13]:
primary_use_map = {'Education':0, \
                   'Lodging/residential':1, \
                   'Office':2,\
                   'Entertainment/public assembly':3,\
                   'Other':4,\
                   'Retail':5,\
                   'Parking':6,\
                   'Public services':7,\
                   'Warehouse/storage':8,\
                   'Food sales and service':9,\
                   'Religious worship':10,\
                   'Healthcare':11,\
                   'Utility':12,\
                   'Technology/science':13,\
                   'Manufacturing/industrial':14,\
                   'Services':15\
                   }

df_building_metadata = pd.read_csv('building_metadata.csv')
df_building_metadata.info()
df_building_metadata.head()

df_building_metadata['use_category'] = df_building_metadata['primary_use'].map(primary_use_map)
df_building_metadata.drop(columns=['primary_use'],inplace=True)
df_building_metadata = reduce_mem_usage(df_building_metadata)
df_building_metadata.info()
df_building_metadata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null object
square_feet    1449 non-null int64
year_built     675 non-null float64
floor_count    355 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.0+ KB


,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


Mem. usage decreased to  0.02 Mb (74.9% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id         1449 non-null int8
building_id     1449 non-null int16
square_feet     1449 non-null int32
year_built      675 non-null float16
floor_count     355 non-null float16
use_category    1449 non-null int8
dtypes: float16(2), int16(1), int32(1), int8(2)
memory usage: 17.1 KB


,site_id,building_id,square_feet,year_built,floor_count,use_category
0,0,0,7432,2008.0,NaN,0
1,0,1,2720,2004.0,NaN,0
2,0,2,5376,1991.0,NaN,0
3,0,3,23685,2002.0,NaN,0
4,0,4,116607,1975.0,NaN,0


In [14]:
print('Missing Values in %:\n{}'.format(100*df_building_metadata.isna().sum()/df_building_metadata.shape[0]))

Missing Values in %:
site_id          0.000000
building_id      0.000000
square_feet      0.000000
year_built      53.416149
floor_count     75.500345
use_category     0.000000
dtype: float64


In [0]:
df_building_metadata.drop(columns=['year_built','floor_count'],inplace=True)

In [16]:
df_building_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 4 columns):
site_id         1449 non-null int8
building_id     1449 non-null int16
square_feet     1449 non-null int32
use_category    1449 non-null int8
dtypes: int16(1), int32(1), int8(2)
memory usage: 11.4 KB


In [0]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

# Weather Train EDA

In [18]:
df_weather_train = pd.read_csv('weather_train.csv',parse_dates = ['timestamp'], date_parser = dateparse)
df_weather_train.info()
df_weather_train.head()
df_weather_train.tail()
df_weather_train = reduce_mem_usage(df_weather_train)
df_weather_train.info()
df_weather_train['day'] = df_weather_train['timestamp'].dt.day
df_weather_train['week'] = df_weather_train['timestamp'].dt.week
df_weather_train['month'] = df_weather_train['timestamp'].dt.month
df_weather_train['hour'] = df_weather_train['timestamp'].dt.hour
df_weather_train.info()
df_weather_train = reduce_mem_usage(df_weather_train)
df_weather_train.info()
df_weather_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int64
timestamp             139773 non-null datetime64[ns]
air_temperature       139718 non-null float64
cloud_coverage        70600 non-null float64
dew_temperature       139660 non-null float64
precip_depth_1_hr     89484 non-null float64
sea_level_pressure    129155 non-null float64
wind_direction        133505 non-null float64
wind_speed            139469 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 9.6 MB


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6
139772,15,2016-12-31 23:00:00,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8


Mem. usage decreased to  3.07 Mb (68.1% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int8
timestamp             139773 non-null datetime64[ns]
air_temperature       139718 non-null float16
cloud_coverage        70600 non-null float16
dew_temperature       139660 non-null float16
precip_depth_1_hr     89484 non-null float16
sea_level_pressure    129155 non-null float16
wind_direction        133505 non-null float16
wind_speed            139469 non-null float16
dtypes: datetime64[ns](1), float16(7), int8(1)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 13 columns):
site_id               139773 non-null int8
timestamp             139773 non-null datetime64[ns]
air_temperature       139718 non-null float16
cloud_coverage        70600 non-null float16
dew_temperature       139660 non-null float16
precip_dept

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,week,month,hour
0,0,2016-01-01 00:00:00,25.000000,6.0,20.00000,NaN,1019.5,0.0,0.000000,1,53,1,0
1,0,2016-01-01 01:00:00,24.406250,NaN,21.09375,-1.0,1020.0,70.0,1.500000,1,53,1,1
2,0,2016-01-01 02:00:00,22.796875,2.0,21.09375,0.0,1020.0,0.0,0.000000,1,53,1,2
3,0,2016-01-01 03:00:00,21.093750,2.0,20.59375,0.0,1020.0,0.0,0.000000,1,53,1,3
4,0,2016-01-01 04:00:00,20.000000,2.0,20.00000,-1.0,1020.0,250.0,2.599609,1,53,1,4


In [19]:
print('Missing Values in %:\n{}'.format(100*df_weather_train.isna().sum()/df_weather_train.shape[0]))

Missing Values in %:
site_id                0.000000
timestamp              0.000000
air_temperature        0.039350
cloud_coverage        49.489529
dew_temperature        0.080845
precip_depth_1_hr     35.979052
sea_level_pressure     7.596603
wind_direction         4.484414
wind_speed             0.217496
day                    0.000000
week                   0.000000
month                  0.000000
hour                   0.000000
dtype: float64


In [0]:
weather_metrics = ['air_temperature', 'cloud_coverage','dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure','wind_direction', 'wind_speed']

In [0]:
def fill_missing_column(df,filler_df,col):
    null_df = df.loc[df[col].isnull()]
    
    if null_df.empty != True:
        null_df[col] = null_df.apply(lambda x: filler_df.loc[x['site_id']][x['month']][x['hour']],axis=1)
        df.loc[null_df.index,col] = null_df[col]
    
    return df


In [22]:
for item in weather_metrics:
  filler = df_weather_train.groupby(by = ['site_id','month','hour'])[item].agg(np.mean)

  df_weather_train = fill_missing_column(df_weather_train,filler,item)
print('Missing Values in %:\n{}'.format(100*df_weather_train.isna().sum()/df_weather_train.shape[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Missing Values in %:
site_id                0.000000
timestamp              0.000000
air_temperature        0.000000
cloud_coverage        12.370057
dew_temperature        0.000000
precip_depth_1_hr     20.325099
sea_level_pressure     6.263728
wind_direction         0.000000
wind_speed             0.000000
day                    0.000000
week                   0.000000
month                  0.000000
hour                   0.000000
dtype: float64


In [23]:
df_weather_train.groupby(['site_id'])['air_temperature'].median()

site_id
0     23.906250
1     11.296875
2     25.593750
3     15.601562
4     15.000000
5     11.000000
6     16.093750
7      7.699219
8     23.906250
9     22.203125
10    11.703125
11     7.699219
12     9.796875
13    11.101562
14    13.296875
15     9.398438
Name: air_temperature, dtype: float16

In [24]:
df_weather_train.groupby(['site_id'])['air_temperature','dew_temperature','sea_level_pressure','cloud_coverage','precip_depth_1_hr','wind_direction', 'wind_speed'].median()

,air_temperature,dew_temperature,sea_level_pressure,cloud_coverage,precip_depth_1_hr,wind_direction,wind_speed
site_id,,,,,,,
0,23.906250,18.296875,1018.0,2.533203,0.0,140.00,3.099609
1,11.296875,7.800781,1017.5,0.000000,NaN,220.00,3.599609
2,25.593750,3.300781,1011.0,2.000000,0.0,130.00,2.599609
3,15.601562,9.398438,1018.0,2.769531,0.0,190.00,3.599609
4,15.000000,10.601562,1016.5,2.000000,0.0,260.00,3.599609
5,11.000000,8.000000,NaN,0.000000,NaN,240.00,4.601562
6,16.093750,8.898438,1017.5,0.000000,0.0,149.25,2.099609
7,7.699219,2.400391,1015.0,NaN,7.0,230.00,3.099609
8,23.906250,18.296875,1018.0,2.533203,0.0,140.00,3.099609


In [25]:
df_weather_train.groupby(['site_id'])['air_temperature','dew_temperature','sea_level_pressure','cloud_coverage','precip_depth_1_hr','wind_direction', 'wind_speed'].mean()

,air_temperature,dew_temperature,sea_level_pressure,cloud_coverage,precip_depth_1_hr,wind_direction,wind_speed
site_id,,,,,,,
0,22.828125,16.812500,1018.0,2.808594,1.382812,156.500,3.375000
1,11.710938,7.648438,1016.0,0.064941,NaN,195.625,3.966797
2,24.937500,4.070312,1011.5,1.906250,0.159180,153.625,2.933594
3,15.523438,8.132812,1018.0,3.277344,0.691406,185.000,3.910156
4,15.234375,9.898438,1017.0,1.936523,0.218994,208.000,3.857422
5,10.867188,8.195312,NaN,0.297119,NaN,200.000,4.769531
6,15.132812,7.312500,1017.5,0.656738,0.797852,132.750,2.199219
7,7.890625,1.891602,1015.5,NaN,13.273438,208.875,3.445312
8,22.828125,16.812500,1018.0,2.808594,1.382812,156.500,3.375000


In [26]:
%env

{'CLICOLOR': '1',
 'CLOUDSDK_CONFIG': '/content/.config',
 'COLAB_GPU': '1',
 'CUDA_PKG_VERSION': '10-0=10.0.130-1',
 'CUDA_VERSION': '10.0.130',
 'CUDNN_VERSION': '7.6.3.30',
 'DATALAB_SETTINGS_OVERRIDES': '{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=\\"172.28.0.2\\""]}',
 'DEBIAN_FRONTEND': 'noninteractive',
 'ENV': '/root/.bashrc',
 'GCS_READ_CACHE_BLOCK_SIZE_MB': '16',
 'GIT_PAGER': 'cat',
 'GLIBCPP_FORCE_NEW': '1',
 'GLIBCXX_FORCE_NEW': '1',
 'HOME': '/root',
 'HOSTNAME': '67b0823fdba4',
 'JPY_PARENT_PID': '19',
 'KMP_DUPLICATE_LIB_OK': 'True',
 'KMP_INIT_AT_FORK': 'FALSE',
 'LANG': 'en_US.UTF-8',
 'LAST_FORCED_REBUILD': '20191018',
 'LD_LIBRARY_PATH': '/usr/lib64-nvidia',
 'LD_PRELOAD': '/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4',
 'LIBRARY_PATH': '/usr/local/cuda/lib64/stubs',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'NCCL_VERSION': '2.4.8',
 'NO_GCE_CHECK': 'True',
 'NVIDIA_DRIVER_CAPABILITIES': 'compute,utility',

In [27]:
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_weather_train', 3774023),
 ('filler', 24283),
 ('df_building_metadata', 11744),
 ('InteractiveShell', 1056),
 ('LinearRegression', 1056),
 ('primary_use_map', 648),
 ('dateparse', 136),
 ('fill_missing_column', 136),
 ('reduce_mem_usage', 136),
 ('weather_metrics', 120),
 ('drive', 80),
 ('lgb', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('item', 59)]

https://www.kaggle.com/datadugong/locate-better-cities-by-weather-temp-fill-nans

	location	corr
site_id		
0	orlando	0.971346
1	heathrow	0.985568
2	Phoenix	0.978950
3	washington	0.990834
4	San Francisco	0.927506
5	birmingham	0.944360
6	washington	0.976488
7	ottowa	0.990955
8	orlando	0.971346
9	sanantonio	0.978205
10	saltlake	0.974286
11	ottowa	0.990955
12	dublin	0.954968
13	Minneapolis	0.994499
14	Philadelphia	0.993035
15	rochester	0.975049




In [28]:
df_weather_train.cloud_coverage.fillna(value=0, inplace=True)
df_weather_train.precip_depth_1_hr.fillna(value=0, inplace=True)
df_weather_train.sea_level_pressure.fillna(value=1015, inplace=True)

df_weather_train.isna().sum()
df_weather_train.groupby(['site_id'])['air_temperature','dew_temperature','sea_level_pressure','cloud_coverage','precip_depth_1_hr','wind_direction', 'wind_speed'].mean()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
day                   0
week                  0
month                 0
hour                  0
dtype: int64

,air_temperature,dew_temperature,sea_level_pressure,cloud_coverage,precip_depth_1_hr,wind_direction,wind_speed
site_id,,,,,,,
0,22.828125,16.812500,1018.0,2.808594,1.382812,156.500,3.375000
1,11.710938,7.648438,1016.0,0.064941,0.000000,195.625,3.966797
2,24.937500,4.070312,1011.5,1.906250,0.159180,153.625,2.933594
3,15.523438,8.132812,1018.0,3.277344,0.691406,185.000,3.910156
4,15.234375,9.898438,1017.0,1.922852,0.218994,208.000,3.857422
5,10.867188,8.195312,1015.0,0.297119,0.000000,200.000,4.769531
6,15.132812,7.312500,1017.5,0.656738,0.797852,132.750,2.199219
7,7.890625,1.891602,1015.5,0.000000,11.828125,208.875,3.445312
8,22.828125,16.812500,1018.0,2.808594,1.382812,156.500,3.375000


# Train Data Manip

In [29]:
df_train = pd.read_csv('train.csv',parse_dates = ['timestamp'], date_parser = dateparse)
df_train.info()
df_train.head()
df_train = reduce_mem_usage(df_train)
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


Mem. usage decreased to 289.19 Mb (53.1% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int16
meter            int8
timestamp        datetime64[ns]
meter_reading    float32
dtypes: datetime64[ns](1), float32(1), int16(1), int8(1)
memory usage: 289.2 MB


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [30]:
df_train = df_train.merge(df_building_metadata,on='building_id')
df_train.info()
df_train = reduce_mem_usage(df_train)
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 7 columns):
building_id      int16
meter            int8
timestamp        datetime64[ns]
meter_reading    float32
site_id          int8
square_feet      int32
use_category     int8
dtypes: datetime64[ns](1), float32(1), int16(1), int32(1), int8(3)
memory usage: 559.1 MB
Mem. usage decreased to 559.11 Mb (0.0% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 7 columns):
building_id      int16
meter            int8
timestamp        datetime64[ns]
meter_reading    float32
site_id          int8
square_feet      int32
use_category     int8
dtypes: datetime64[ns](1), float32(1), int16(1), int32(1), int8(3)
memory usage: 559.1 MB


,building_id,meter,timestamp,meter_reading,site_id,square_feet,use_category
0,0,0,2016-01-01 00:00:00,0.0,0,7432,0
1,0,0,2016-01-01 01:00:00,0.0,0,7432,0
2,0,0,2016-01-01 02:00:00,0.0,0,7432,0
3,0,0,2016-01-01 03:00:00,0.0,0,7432,0
4,0,0,2016-01-01 04:00:00,0.0,0,7432,0


In [31]:
df_train = df_train.merge(df_weather_train,on=['site_id','timestamp'])
df_train.info()
df_train = reduce_mem_usage(df_train)
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 18 columns):
building_id           int16
meter                 int8
timestamp             datetime64[ns]
meter_reading         float32
site_id               int8
square_feet           int32
use_category          int8
air_temperature       float16
cloud_coverage        float16
dew_temperature       float16
precip_depth_1_hr     float16
sea_level_pressure    float16
wind_direction        float16
wind_speed            float16
day                   int8
week                  int8
month                 int8
hour                  int8
dtypes: datetime64[ns](1), float16(7), float32(1), int16(1), int32(1), int8(7)
memory usage: 902.1 MB
Mem. usage decreased to 902.08 Mb (0.0% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 18 columns):
building_id           int16
meter                 int8
timestamp             datetime64[ns]
met

,building_id,meter,timestamp,meter_reading,site_id,square_feet,use_category,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,week,month,hour
0,0,0,2016-01-01,0.0,0,7432,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0
1,1,0,2016-01-01,0.0,0,2720,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0
2,2,0,2016-01-01,0.0,0,5376,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0
3,3,0,2016-01-01,0.0,0,23685,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0
4,4,0,2016-01-01,0.0,0,116607,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0


In [32]:
df_train.isna().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
square_feet           0
use_category          0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
day                   0
week                  0
month                 0
hour                  0
dtype: int64

In [33]:
df_train.drop(columns=['timestamp'],inplace=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 17 columns):
building_id           int16
meter                 int8
meter_reading         float32
site_id               int8
square_feet           int32
use_category          int8
air_temperature       float16
cloud_coverage        float16
dew_temperature       float16
precip_depth_1_hr     float16
sea_level_pressure    float16
wind_direction        float16
wind_speed            float16
day                   int8
week                  int8
month                 int8
hour                  int8
dtypes: float16(7), float32(1), int16(1), int32(1), int8(7)
memory usage: 748.5 MB


In [34]:
df_train['log_meter_reading'] = np.log1p(df_train['meter_reading'])
df_train.drop(columns=['meter_reading'],inplace=True) 

df_train.info()
df_train = reduce_mem_usage(df_train)
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 17 columns):
building_id           int16
meter                 int8
site_id               int8
square_feet           int32
use_category          int8
air_temperature       float16
cloud_coverage        float16
dew_temperature       float16
precip_depth_1_hr     float16
sea_level_pressure    float16
wind_direction        float16
wind_speed            float16
day                   int8
week                  int8
month                 int8
hour                  int8
log_meter_reading     float32
dtypes: float16(7), float32(1), int16(1), int32(1), int8(7)
memory usage: 748.5 MB
Mem. usage decreased to 710.15 Mb (5.1% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125605 entries, 0 to 20125604
Data columns (total 17 columns):
building_id           int16
meter                 int8
site_id               int8
square_feet           int32
use_category          int8
air_temperatur

,building_id,meter,site_id,square_feet,use_category,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,week,month,hour,log_meter_reading
0,0,0,0,7432,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0,0.0
1,1,0,0,2720,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0,0.0
2,2,0,0,5376,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0,0.0
3,3,0,0,23685,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0,0.0
4,4,0,0,116607,0,25.0,6.0,20.0,2.433594,1019.5,0.0,0.0,1,53,1,0,0.0


In [35]:
# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_train', 744647409),
 ('df_weather_train', 3774023),
 ('filler', 24283),
 ('df_building_metadata', 11744),
 ('InteractiveShell', 1056),
 ('LinearRegression', 1056),
 ('primary_use_map', 648),
 ('dateparse', 136),
 ('fill_missing_column', 136),
 ('reduce_mem_usage', 136),
 ('weather_metrics', 120),
 ('drive', 80),
 ('lgb', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('item', 59)]

In [36]:
!pip install tqdm

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [0]:
all_features = [col for col in df_train.columns if col not in ['site_id', 'meter', 'log_meter_reading']]
categorical_features = ['building_id', 'use_category', 'day', 'week', 'month', 'hour' ]

In [0]:
cv = 3
seed = 42
models = {'site_id': [], 'meter': [], 'model': []}
cv_scores = {"site_id": [], "meter": [], "cv_score": []}

In [79]:
for site in tqdm(range(16), desc="site_id"):

  for meter_id in tqdm(range(4), desc="meter_id"):
    print("{} fold CV for site_id:{}, meter_id:{}".format(cv, site, meter_id))
    kf = KFold(n_splits=cv, random_state=seed)
    
    

    X_train_site_meter = df_train.query('site_id==@site & meter==@meter_id').reset_index(drop=True)
    y_train_site_meter = X_train_site_meter.log_meter_reading
    y_pred_train_site_meter = np.zeros(X_train_site_meter.shape[0])

    score = 0

    if X_train_site_meter.shape[0] != 0:
      for fold, (train_index, valid_index) in enumerate(kf.split(X_train_site_meter, y_train_site_meter)):
              X_train, X_valid = X_train_site_meter.loc[train_index, all_features], X_train_site_meter.loc[valid_index, all_features]
              y_train, y_valid = y_train_site_meter.iloc[train_index], y_train_site_meter.iloc[valid_index]

              dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
              dvalid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=categorical_features)

              watchlist = [dtrain, dvalid]

              params = {"objective": "regression",
                        "num_leaves": 40,
                        "learning_rate": 0.05,
                        "bagging_fraction": 0.9,
                        "feature_fraction": 0.85,
                        "metric": "rmse"
                        }

              model_lgb = lgb.train(params, train_set=dtrain, num_boost_round=1000, valid_sets=watchlist, verbose_eval=200, early_stopping_rounds=200)
              models['site_id'].append(site)
              models['meter'].append(meter_id)
              models['model'].append(model_lgb)

              y_pred_valid = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
              y_pred_train_site_meter[valid_index] = y_pred_valid

              rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
              print("Site Id: {}, Meter: {}, Fold: {}, RMSE: {}".format(site, meter_id, fold+1,rmse))
              score += rmse / cv

              gc.collect()

      cv_scores["site_id"].append(site)
      cv_scores["meter"].append(meter_id)
      cv_scores["cv_score"].append(score)
          
      print("\nSite Id: {}, Meter: {}, CV RMSE: {} \n".format(site, meter_id, np.sqrt(mean_squared_error(y_train_site_meter, y_pred_train_site_meter))))     









site_id:   0%|          | 0/16 [00:00<?, ?it/s]







meter_id:   0%|          | 0/4 [00:00<?, ?it/s]

3 fold CV for site_id:0, meter_id:0


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[200]	training's rmse: 0.24397	valid_1's rmse: 4.14923
Early stopping, best iteration is:
[162]	training's rmse: 0.254129	valid_1's rmse: 4.1452
Site Id: 0, Meter: 0, Fold: 1, RMSE: 5.315967319286872


267

...

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[200]	training's rmse: 0.197881	valid_1's rmse: 0.669787
[400]	training's rmse: 0.160875	valid_1's rmse: 0.670508
Early stopping, best iteration is:
[325]	training's rmse: 0.172382	valid_1's rmse: 0.669482
Site Id: 15, Meter: 3, Fold: 3, RMSE: 0.8729131377805395


296









meter_id: 100%|██████████| 4/4 [01:43<00:00, 25.70s/it]














site_id: 100%|██████████| 16/16 [13:55<00:00, 71.78s/it]









Site Id: 15, Meter: 3, CV RMSE: 0.7868986966104192 



In [80]:
cv_scores

{'cv_score': [3.935358013260487,
  2.13305276402256,
  0.6771313997579412,
  2.4115705561653846,
  0.5661726719176968,
  1.4138618010831552,
  1.3610703594632192,
  0.4761444395398905,
  0.2898011878825949,
  0.8033793930656978,
  0.2631078289468036,
  2.564007537864365,
  2.196832547468289,
  2.0983602135185624,
  2.6751453714805917,
  3.4577236977777077,
  3.797742068890129,
  0.5618056588682785,
  1.025155914192542,
  1.480441655450357,
  1.5215287304480047,
  0.5002407653094562,
  1.7981621618827819,
  3.1247864603800166,
  0.3869636524739717,
  2.705638145191532,
  2.2653732829840876,
  0.4367264979557901,
  0.6182618700505748,
  1.9603613024294908,
  2.0794999355856474,
  1.0565531242969033,
  1.8948528330254182,
  1.8396060279611977,
  2.0920275397309998,
  0.3592862107591214,
  0.8290538006340553,
  1.2489493070701385,
  0.7828957651346924],
 'meter': [0,
  1,
  0,
  3,
  0,
  1,
  3,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
  1,
  2,
  3,
  0,
  0,
  1,
  2,
  0,
  1,
  3,
  0,
  1,

In [85]:
type(models)
len(models['model'])

dict

117

In [69]:
cv_scores

{'cv_score': [3.8864075692567397,
  0.6771313997579412,
  0.5671451033764106,
  0.4790531736622141,
  0.2897623150712714,
  3.8864075692567397,
  0.6771313997579412],
 'meter': [0, 0, 0, 0, 0, 0, 0],
 'site_id': [0, 1, 2, 3, 4, 0, 1]}

In [0]:
for site_id in tqdm(range(1), desc="site_id"):
  for meter in tqdm(range(4), desc='meter'):


    print(cv, "fold CV for site_id:", site_id)
    kf = KFold(n_splits=cv, random_state=seed)
    models[site_id] = []

    X_train_site = df_train[df_train.site_id==site_id & df_train.meter==meter].reset_index(drop=True)
    y_train_site = X_train_site.log_meter_reading
    y_pred_train_site = np.zeros(X_train_site.shape[0])

# LGB Model

In [0]:
cv = 3
seed = 42
models = {}
cv_scores = {"site_id": [], "cv_score": []}

for site_id in tqdm(range(16), desc="site_id"):
    print(cv, "fold CV for site_id:", site_id)
    kf = KFold(n_splits=cv, random_state=seed)
    models[site_id] = []

    X_train_site = df_train[df_train.site_id==site_id].reset_index(drop=True)
    y_train_site = X_train_site.log_meter_reading
    y_pred_train_site = np.zeros(X_train_site.shape[0])   

    score = 0


    for fold, (train_index, valid_index) in enumerate(kf.split(X_train_site, y_train_site)):
            X_train, X_valid = X_train_site.loc[train_index, all_features], X_train_site.loc[valid_index, all_features]
            y_train, y_valid = y_train_site.iloc[train_index], y_train_site.iloc[valid_index]

            dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
            dvalid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=categorical_features)

            watchlist = [dtrain, dvalid]

            params = {"objective": "regression",
                      "num_leaves": 40,
                      "learning_rate": 0.05,
                      "bagging_fraction": 0.9,
                      "feature_fraction": 0.85,
                      "metric": "rmse"
                      }

            model_lgb = lgb.train(params, train_set=dtrain, num_boost_round=2000, valid_sets=watchlist, verbose_eval=200, early_stopping_rounds=200)
            models[site_id].append(model_lgb)

            y_pred_valid = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
            y_pred_train_site[valid_index] = y_pred_valid

            rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
            print("Site Id:", site_id, ", Fold:", fold+1, ", RMSE:", rmse)
            score += rmse / cv

            gc.collect()

    cv_scores["site_id"].append(site_id)
    cv_scores["cv_score"].append(score)
        
    print("\nSite Id:", site_id, ", CV RMSE:", np.sqrt(mean_squared_error(y_train_site, y_pred_train_site)), "\n")            


site_id:   0%|          | 0/16 [00:00<?, ?it/s]

3 fold CV for site_id: 0


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[200]	training's rmse: 0.467245	valid_1's rmse: 4.49038
[400]	training's rmse: 0.410099	valid_1's rmse: 4.49347
Early stopping, best iteration is:
[208]	training's rmse: 0.464324	valid_1's rmse: 4.48955
Site Id: 0 , Fold: 1 , RMSE: 5.061626620195897


39

...

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[200]	training's rmse: 0.520632	valid_1's rmse: 0.727915
Early stopping, best iteration is:
[94]	training's rmse: 0.588954	valid_1's rmse: 0.72028
Site Id: 15 , Fold: 3 , RMSE: 0.8096948126898141


483

site_id: 100%|██████████| 16/16 [20:22<00:00, 118.60s/it]


Site Id: 15 , CV RMSE: 0.8416337185866807 



In [86]:
pd.DataFrame.from_dict(cv_scores)

,site_id,meter,cv_score
0,0,0,3.935358
1,0,1,2.133053
2,1,0,0.677131
3,1,3,2.411571
4,2,0,0.566173
5,2,1,1.413862
6,2,3,1.361070
7,3,0,0.476144
8,4,0,0.289801
9,5,0,0.803379


In [87]:
# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_train', 744647409),
 ('df_weather_train', 3774023),
 ('X_train_site_meter', 412416),
 ('X_train', 312798),
 ('X_valid', 156378),
 ('y_pred_train_site_meter', 113824),
 ('y_train', 94804),
 ('train_index', 75920),
 ('y_valid', 47404),
 ('valid_index', 38000),
 ('y_pred_valid', 38000),
 ('y_train_site_meter', 28584),
 ('filler', 24327),
 ('df_building_metadata', 11744),
 ('InteractiveShell', 1056),
 ('KFold', 1056),
 ('LinearRegression', 1056),
 ('primary_use_map', 648),
 ('params', 368),
 ('cv_scores', 240),
 ('models', 240),
 ('all_features', 192),
 ('dateparse', 136),
 ('fill_missing_column', 136),
 ('mean_squared_error', 136),
 ('reduce_mem_usage', 136),
 ('weather_metrics', 120),
 ('categorical_features', 112),
 ('drive', 80),
 ('lgb', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('watchlist', 80),
 ('item', 59),
 ('dtrain', 56),
 ('dvalid', 56),
 ('kf', 56),
 ('model_lgb', 56),
 ('rmse', 32),
 ('score', 32),
 ('cv', 28),
 ('fold', 28),
 ('meter', 28),
 ('meter_

In [88]:
del df_train, X_train, X_valid, X_train_site_meter, y_pred_train_site_meter, y_train, y_valid, y_train_site_meter, train_index, valid_index
gc.collect()

831

# Weather Test EDA

In [89]:
df_weather_test = pd.read_csv('weather_test.csv',parse_dates = ['timestamp'], date_parser = dateparse)
df_weather_test.info()
df_weather_test.head()
df_weather_test.tail()
df_weather_test = reduce_mem_usage(df_weather_test)
df_weather_test.info()
df_weather_test['day'] = df_weather_test['timestamp'].dt.day
df_weather_test['week'] = df_weather_test['timestamp'].dt.week
df_weather_test['month'] = df_weather_test['timestamp'].dt.month
df_weather_test['hour'] = df_weather_test['timestamp'].dt.hour
df_weather_test.info()
df_weather_test = reduce_mem_usage(df_weather_test)
df_weather_test.info()
df_weather_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277243 entries, 0 to 277242
Data columns (total 9 columns):
site_id               277243 non-null int64
timestamp             277243 non-null datetime64[ns]
air_temperature       277139 non-null float64
cloud_coverage        136795 non-null float64
dew_temperature       276916 non-null float64
precip_depth_1_hr     181655 non-null float64
sea_level_pressure    255978 non-null float64
wind_direction        264873 non-null float64
wind_speed            276783 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 19.0 MB


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
277238,15,2018-12-31 19:00:00,3.3,NaN,1.7,NaN,1018.3,150.0,7.7
277239,15,2018-12-31 20:00:00,2.8,NaN,1.1,NaN,1017.8,140.0,5.1
277240,15,2018-12-31 21:00:00,2.8,NaN,1.7,-1.0,1017.2,140.0,6.2
277241,15,2018-12-31 22:00:00,2.8,NaN,2.2,8.0,1016.1,140.0,5.1
277242,15,2018-12-31 23:00:00,3.3,NaN,2.2,20.0,1014.7,140.0,5.1


Mem. usage decreased to  6.08 Mb (68.1% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277243 entries, 0 to 277242
Data columns (total 9 columns):
site_id               277243 non-null int8
timestamp             277243 non-null datetime64[ns]
air_temperature       277139 non-null float16
cloud_coverage        136795 non-null float16
dew_temperature       276916 non-null float16
precip_depth_1_hr     181655 non-null float16
sea_level_pressure    255978 non-null float16
wind_direction        264873 non-null float16
wind_speed            276783 non-null float16
dtypes: datetime64[ns](1), float16(7), int8(1)
memory usage: 6.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277243 entries, 0 to 277242
Data columns (total 13 columns):
site_id               277243 non-null int8
timestamp             277243 non-null datetime64[ns]
air_temperature       277139 non-null float16
cloud_coverage        136795 non-null float16
dew_temperature       276916 non-null float16
precip_d

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,week,month,hour
0,0,2017-01-01 00:00:00,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609,1,52,1,0
1,0,2017-01-01 01:00:00,17.796875,2.0,12.796875,0.0,1022.0,130.0,3.099609,1,52,1,1
2,0,2017-01-01 02:00:00,16.093750,0.0,12.796875,0.0,1022.0,140.0,3.099609,1,52,1,2
3,0,2017-01-01 03:00:00,17.203125,0.0,13.296875,0.0,1022.0,140.0,3.099609,1,52,1,3
4,0,2017-01-01 04:00:00,16.703125,2.0,13.296875,0.0,1022.5,130.0,2.599609,1,52,1,4


In [90]:
print('Missing Values in %:\n{}'.format(100*df_weather_test.isna().sum()/df_weather_test.shape[0]))

Missing Values in %:
site_id                0.000000
timestamp              0.000000
air_temperature        0.037512
cloud_coverage        50.658808
dew_temperature        0.117947
precip_depth_1_hr     34.478057
sea_level_pressure     7.670167
wind_direction         4.461790
wind_speed             0.165919
day                    0.000000
week                   0.000000
month                  0.000000
hour                   0.000000
dtype: float64


In [91]:
for item in weather_metrics:
  filler = df_weather_test.groupby(by = ['site_id','month','hour'])[item].agg(np.mean)

  df_weather_test = fill_missing_column(df_weather_test,filler,item)
print('Missing Values in %:\n{}'.format(100*df_weather_test.isna().sum()/df_weather_test.shape[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Missing Values in %:
site_id                0.000000
timestamp              0.000000
air_temperature        0.000000
cloud_coverage        11.955577
dew_temperature        0.000000
precip_depth_1_hr     18.900748
sea_level_pressure     6.218732
wind_direction         0.000000
wind_speed             0.000000
day                    0.000000
week                   0.000000
month                  0.000000
hour                   0.000000
dtype: float64


In [92]:
df_weather_test.groupby(['site_id'])['air_temperature'].median()

site_id
0     23.906250
1     11.796875
2     25.000000
3     16.093750
4     14.398438
5     11.000000
6     16.093750
7      7.699219
8     23.906250
9     22.796875
10    10.601562
11     7.699219
12    10.101562
13     8.898438
14    13.296875
15     9.398438
Name: air_temperature, dtype: float16

In [93]:
df_weather_test.groupby(['site_id'])['air_temperature','dew_temperature','sea_level_pressure','cloud_coverage','precip_depth_1_hr','wind_direction', 'wind_speed'].median()
df_weather_test.groupby(['site_id'])['air_temperature','dew_temperature','sea_level_pressure','cloud_coverage','precip_depth_1_hr','wind_direction', 'wind_speed'].mean()

,air_temperature,dew_temperature,sea_level_pressure,cloud_coverage,precip_depth_1_hr,wind_direction,wind_speed
site_id,,,,,,,
0,23.906250,19.406250,1018.0,2.500000,0.000000,136.750,3.099609
1,11.796875,8.000000,1016.5,0.000000,NaN,220.000,3.599609
2,25.000000,2.800781,1010.5,2.000000,0.000000,132.000,2.599609
3,16.093750,10.000000,1018.0,2.572266,0.000000,190.000,3.599609
4,14.398438,10.601562,1016.5,2.000000,0.000000,260.000,3.599609
5,11.000000,9.000000,NaN,0.000000,NaN,250.000,4.601562
6,16.093750,10.000000,1017.5,0.000000,0.000000,140.000,2.099609
7,7.699219,2.599609,1015.5,NaN,8.859375,220.000,3.099609
8,23.906250,19.406250,1018.0,2.500000,0.000000,136.750,3.099609


,air_temperature,dew_temperature,sea_level_pressure,cloud_coverage,precip_depth_1_hr,wind_direction,wind_speed
site_id,,,,,,,
0,22.703125,17.656250,1018.0,2.816406,1.168945,150.750,3.400391
1,12.171875,7.542969,1015.5,0.050964,NaN,198.875,4.039062
2,24.968750,3.816406,1011.5,2.080078,0.153931,155.125,2.882812
3,15.492188,8.367188,1018.0,3.371094,0.997559,184.000,3.902344
4,14.468750,9.804688,1017.0,1.873047,0.516602,204.375,3.753906
5,11.156250,8.437500,NaN,0.496094,NaN,209.875,4.886719
6,15.234375,7.867188,1017.5,0.695312,1.147461,128.625,2.257812
7,7.406250,2.109375,1016.0,NaN,13.757812,206.000,3.361328
8,22.703125,17.656250,1018.0,2.816406,1.168945,150.750,3.400391


In [94]:
# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_weather_test', 7485713),
 ('df_weather_train', 3774023),
 ('y_pred_valid', 38000),
 ('filler', 24327),
 ('df_building_metadata', 11744),
 ('InteractiveShell', 1056),
 ('KFold', 1056),
 ('LinearRegression', 1056),
 ('primary_use_map', 648),
 ('params', 368),
 ('cv_scores', 240),
 ('models', 240),
 ('all_features', 192),
 ('dateparse', 136),
 ('fill_missing_column', 136),
 ('mean_squared_error', 136),
 ('reduce_mem_usage', 136),
 ('weather_metrics', 120),
 ('categorical_features', 112),
 ('drive', 80),
 ('lgb', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('watchlist', 80),
 ('item', 59),
 ('dtrain', 56),
 ('dvalid', 56),
 ('kf', 56),
 ('model_lgb', 56),
 ('rmse', 32),
 ('score', 32),
 ('cv', 28),
 ('fold', 28),
 ('meter', 28),
 ('meter_id', 28),
 ('seed', 28),
 ('site_id', 28)]

In [95]:
df_weather_test.cloud_coverage.fillna(value=0, inplace=True)
df_weather_test.precip_depth_1_hr.fillna(value=0, inplace=True)
df_weather_test.sea_level_pressure.fillna(value=1015, inplace=True)

df_weather_test.isna().sum()
df_weather_test.groupby(['site_id'])['air_temperature','dew_temperature','sea_level_pressure','cloud_coverage','precip_depth_1_hr','wind_direction', 'wind_speed'].mean()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
day                   0
week                  0
month                 0
hour                  0
dtype: int64

,air_temperature,dew_temperature,sea_level_pressure,cloud_coverage,precip_depth_1_hr,wind_direction,wind_speed
site_id,,,,,,,
0,22.703125,17.656250,1018.0,2.816406,1.168945,150.750,3.400391
1,12.171875,7.542969,1015.5,0.050964,0.000000,198.875,4.039062
2,24.968750,3.816406,1011.5,2.080078,0.153931,155.125,2.882812
3,15.492188,8.367188,1018.0,3.371094,0.997559,184.000,3.902344
4,14.468750,9.804688,1017.0,1.873047,0.516602,204.375,3.753906
5,11.156250,8.437500,1015.0,0.496094,0.000000,209.875,4.886719
6,15.234375,7.867188,1017.5,0.695312,1.147461,128.625,2.257812
7,7.406250,2.109375,1016.0,0.000000,13.507812,206.000,3.361328
8,22.703125,17.656250,1018.0,2.816406,1.168945,150.750,3.400391


# Test Data Manip

In [96]:
df_test = pd.read_csv('test.csv',parse_dates = ['timestamp'], date_parser = dateparse)
df_test.info()
df_test.head()
df_test = reduce_mem_usage(df_test)
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
row_id         int64
building_id    int64
meter          int64
timestamp      datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.2 GB


,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


Mem. usage decreased to 596.49 Mb (53.1% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
row_id         int32
building_id    int16
meter          int8
timestamp      datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(1), int8(1)
memory usage: 596.5 MB


,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


In [97]:
df_test = df_test.merge(df_building_metadata,on='building_id')
df_test.info()
df_test = reduce_mem_usage(df_test)
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 7 columns):
row_id          int32
building_id     int16
meter           int8
timestamp       datetime64[ns]
site_id         int8
square_feet     int32
use_category    int8
dtypes: datetime64[ns](1), int16(1), int32(2), int8(3)
memory usage: 1.1 GB
Mem. usage decreased to 1153.21 Mb (0.0% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 7 columns):
row_id          int32
building_id     int16
meter           int8
timestamp       datetime64[ns]
site_id         int8
square_feet     int32
use_category    int8
dtypes: datetime64[ns](1), int16(1), int32(2), int8(3)
memory usage: 1.1 GB


,row_id,building_id,meter,timestamp,site_id,square_feet,use_category
0,0,0,0,2017-01-01 00:00:00,0,7432,0
1,129,0,0,2017-01-01 01:00:00,0,7432,0
2,258,0,0,2017-01-01 02:00:00,0,7432,0
3,387,0,0,2017-01-01 03:00:00,0,7432,0
4,516,0,0,2017-01-01 04:00:00,0,7432,0


In [98]:
df_test.info()
df_test = reduce_mem_usage(df_test)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 7 columns):
row_id          int32
building_id     int16
meter           int8
timestamp       datetime64[ns]
site_id         int8
square_feet     int32
use_category    int8
dtypes: datetime64[ns](1), int16(1), int32(2), int8(3)
memory usage: 1.1 GB
Mem. usage decreased to 1153.21 Mb (0.0% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 7 columns):
row_id          int32
building_id     int16
meter           int8
timestamp       datetime64[ns]
site_id         int8
square_feet     int32
use_category    int8
dtypes: datetime64[ns](1), int16(1), int32(2), int8(3)
memory usage: 1.1 GB


In [99]:
df_test = df_test.merge(df_weather_test,on=['site_id','timestamp'],how='left')

                      
df_test.info()
df_test = reduce_mem_usage(df_test)
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 18 columns):
row_id                int32
building_id           int16
meter                 int8
timestamp             datetime64[ns]
site_id               int8
square_feet           int32
use_category          int8
air_temperature       float16
cloud_coverage        float16
dew_temperature       float16
precip_depth_1_hr     float16
sea_level_pressure    float16
wind_direction        float16
wind_speed            float16
day                   float64
week                  float64
month                 float64
hour                  float64
dtypes: datetime64[ns](1), float16(7), float64(4), int16(1), int32(2), int8(3)
memory usage: 2.9 GB
Mem. usage decreased to 2028.06 Mb (32.0% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 18 columns):
row_id                int32
building_id           int16
meter                 int8
ti

,row_id,building_id,meter,timestamp,site_id,square_feet,use_category,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,week,month,hour
0,0,0,0,2017-01-01 00:00:00,0,7432,0,17.796875,4.0,11.703125,0.116638,1021.5,100.0,3.599609,1.0,52.0,1.0,0.0
1,129,0,0,2017-01-01 01:00:00,0,7432,0,17.796875,2.0,12.796875,0.000000,1022.0,130.0,3.099609,1.0,52.0,1.0,1.0
2,258,0,0,2017-01-01 02:00:00,0,7432,0,16.093750,0.0,12.796875,0.000000,1022.0,140.0,3.099609,1.0,52.0,1.0,2.0
3,387,0,0,2017-01-01 03:00:00,0,7432,0,17.203125,0.0,13.296875,0.000000,1022.0,140.0,3.099609,1.0,52.0,1.0,3.0
4,516,0,0,2017-01-01 04:00:00,0,7432,0,16.703125,2.0,13.296875,0.000000,1022.5,130.0,2.599609,1.0,52.0,1.0,4.0


In [100]:
df_test.isna().sum()

row_id                     0
building_id                0
meter                      0
timestamp                  0
site_id                    0
square_feet                0
use_category               0
air_temperature       199029
cloud_coverage        199029
dew_temperature       199029
precip_depth_1_hr     199029
sea_level_pressure    199029
wind_direction        199029
wind_speed            199029
day                   199029
week                  199029
month                 199029
hour                  199029
dtype: int64

In [103]:
df_test[df_test.isna()]

,row_id,building_id,meter,site_id,square_feet,use_category,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,week,month,hour
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41697595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41697596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41697597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41697598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
df_test.drop(columns=['timestamp'],inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 17 columns):
row_id                int32
building_id           int16
meter                 int8
site_id               int8
square_feet           int32
use_category          int8
air_temperature       float16
cloud_coverage        float16
dew_temperature       float16
precip_depth_1_hr     float16
sea_level_pressure    float16
wind_direction        float16
wind_speed            float16
day                   float16
week                  float16
month                 float16
hour                  float16
dtypes: float16(11), int16(1), int32(2), int8(3)
memory usage: 1.7 GB


In [102]:
# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_test', 1792996824),
 ('df_weather_test', 7485713),
 ('df_weather_train', 3774023),
 ('y_pred_valid', 38000),
 ('filler', 24327),
 ('df_building_metadata', 11744),
 ('InteractiveShell', 1056),
 ('KFold', 1056),
 ('LinearRegression', 1056),
 ('primary_use_map', 648),
 ('params', 368),
 ('cv_scores', 240),
 ('models', 240),
 ('all_features', 192),
 ('dateparse', 136),
 ('fill_missing_column', 136),
 ('mean_squared_error', 136),
 ('reduce_mem_usage', 136),
 ('weather_metrics', 120),
 ('categorical_features', 112),
 ('drive', 80),
 ('lgb', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('watchlist', 80),
 ('item', 59),
 ('dtrain', 56),
 ('dvalid', 56),
 ('kf', 56),
 ('model_lgb', 56),
 ('rmse', 32),
 ('score', 32),
 ('cv', 28),
 ('fold', 28),
 ('meter', 28),
 ('meter_id', 28),
 ('seed', 28),
 ('site_id', 28)]

# Final

In [0]:
for site in tqdm(range(16), desc="site_id"):

  for meter_id in tqdm(range(4), desc="meter_id"):
    print("{} fold CV for site_id:{}, meter_id:{}".format(cv, site, meter_id))
    kf = KFold(n_splits=cv, random_state=seed)
    
    

    X_train_site_meter = df_train.query('site_id==@site & meter==@meter_id').reset_index(drop=True)
    y_train_site_meter = X_train_site_meter.log_meter_reading
    y_pred_train_site_meter = np.zeros(X_train_site_meter.shape[0])

    score = 0

    if X_train_site_meter.shape[0] != 0:
      for fold, (train_index, valid_index) in enumerate(kf.split(X_train_site_meter, y_train_site_meter)):
              X_train, X_valid = X_train_site_meter.loc[train_index, all_features], X_train_site_meter.loc[valid_index, all_features]
              y_train, y_valid = y_train_site_meter.iloc[train_index], y_train_site_meter.iloc[valid_index]

              dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
              dvalid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=categorical_features)

              watchlist = [dtrain, dvalid]

              params = {"objective": "regression",
                        "num_leaves": 40,
                        "learning_rate": 0.05,
                        "bagging_fraction": 0.9,
                        "feature_fraction": 0.85,
                        "metric": "rmse"
                        }

              model_lgb = lgb.train(params, train_set=dtrain, num_boost_round=1000, valid_sets=watchlist, verbose_eval=200, early_stopping_rounds=200)
              models['site_id'].append(site)
              models['meter'].append(meter_id)
              models['model'].append(model_lgb)

              y_pred_valid = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
              y_pred_train_site_meter[valid_index] = y_pred_valid

              rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
              print("Site Id: {}, Meter: {}, Fold: {}, RMSE: {}".format(site, meter_id, fold+1,rmse))
              score += rmse / cv

              gc.collect()

      cv_scores["site_id"].append(site)
      cv_scores["meter"].append(meter_id)
      cv_scores["cv_score"].append(score)
          
      print("\nSite Id: {}, Meter: {}, CV RMSE: {} \n".format(site, meter_id, np.sqrt(mean_squared_error(y_train_site_meter, y_pred_train_site_meter))))     


In [0]:
    X_train_site_meter = df_train.query('site_id==@site & meter==@meter_id').reset_index(drop=True)
    y_train_site_meter = X_train_site_meter.log_meter_reading
    y_pred_train_site_meter = np.zeros(X_train_site_meter.shape[0])

In [104]:
df_test.head()

,row_id,building_id,meter,site_id,square_feet,use_category,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,week,month,hour
0,0,0,0,0,7432,0,17.796875,4.0,11.703125,0.116638,1021.5,100.0,3.599609,1.0,52.0,1.0,0.0
1,129,0,0,0,7432,0,17.796875,2.0,12.796875,0.000000,1022.0,130.0,3.099609,1.0,52.0,1.0,1.0
2,258,0,0,0,7432,0,16.093750,0.0,12.796875,0.000000,1022.0,140.0,3.099609,1.0,52.0,1.0,2.0
3,387,0,0,0,7432,0,17.203125,0.0,13.296875,0.000000,1022.0,140.0,3.099609,1.0,52.0,1.0,3.0
4,516,0,0,0,7432,0,16.703125,2.0,13.296875,0.000000,1022.5,130.0,2.599609,1.0,52.0,1.0,4.0


In [144]:
model_df = pd.DataFrame(models)
model_df.head(20)

,site_id,meter,model
0,0,0,<lightgbm.basic.Booster object at 0x7f03f1e23cf8>
1,0,0,<lightgbm.basic.Booster object at 0x7f03f1e23d30>
2,0,0,<lightgbm.basic.Booster object at 0x7f03f1e2c278>
3,0,1,<lightgbm.basic.Booster object at 0x7f03f222a0f0>
4,0,1,<lightgbm.basic.Booster object at 0x7f03f1e88b00>
5,0,1,<lightgbm.basic.Booster object at 0x7f03f1e88940>
6,1,0,<lightgbm.basic.Booster object at 0x7f03f208bd68>
7,1,0,<lightgbm.basic.Booster object at 0x7f03f2302278>
8,1,0,<lightgbm.basic.Booster object at 0x7f03f1e23a90>
9,1,3,<lightgbm.basic.Booster object at 0x7f03f1f302b0>


In [158]:
list(model_df.query('site_id == 0 & meter == 1')['model'])[0]

In [159]:
for site in range(16):
  for meter_id in range(4):
    if model_df.query('site_id == @site & meter == @meter_id').shape[0] != 0:
      print(list(model_df.query('site_id == @site & meter == @meter_id')['model']))


[<lightgbm.basic.Booster object at 0x7f03f1e23cf8>, <lightgbm.basic.Booster object at 0x7f03f1e23d30>, <lightgbm.basic.Booster object at 0x7f03f1e2c278>]
[<lightgbm.basic.Booster object at 0x7f03f222a0f0>, <lightgbm.basic.Booster object at 0x7f03f1e88b00>, <lightgbm.basic.Booster object at 0x7f03f1e88940>]
[<lightgbm.basic.Booster object at 0x7f03f208bd68>, <lightgbm.basic.Booster object at 0x7f03f2302278>, <lightgbm.basic.Booster object at 0x7f03f1e23a90>]
[<lightgbm.basic.Booster object at 0x7f03f1f302b0>, <lightgbm.basic.Booster object at 0x7f03f1e88128>, <lightgbm.basic.Booster object at 0x7f03f1e882b0>]
[<lightgbm.basic.Booster object at 0x7f03f1fdaeb8>, <lightgbm.basic.Booster object at 0x7f03f1e23160>, <lightgbm.basic.Booster object at 0x7f03f1ea4c18>]
[<lightgbm.basic.Booster object at 0x7f03f1e2c400>, <lightgbm.basic.Booster object at 0x7f03f1e2cdd8>, <lightgbm.basic.Booster object at 0x7f03f1e884e0>]
[<lightgbm.basic.Booster object at 0x7f03f229ea58>, <lightgbm.basic.Booster 

In [160]:
df_test_sites_meters = []

for site in tqdm(range(16), desc="site_id"):
  for meter_id in tqdm(range(4), desc='meter_id'):
    print("Preparing test data for site_id", site_id)

    if model_df.query('site_id == @site & meter == @meter_id').shape[0] != 0:

      X_test_site_meter = df_test.query('site_id== @site & meter == @meter_id')
      
      row_ids_site_meter = X_test_site_meter.row_id

      X_test_site_meter = X_test_site_meter[all_features]
      y_pred_test_site_meter = np.zeros(X_test_site_meter.shape[0])

      print("Scoring for site_id: {}, meter: {}".format(site, meter_id))    
      for fold in range(cv):
          model_lgb = list(model_df.query('site_id == @site & meter == @meter_id')['model'])[fold]
          y_pred_test_site_meter += model_lgb.predict(X_test_site_meter, num_iteration=model_lgb.best_iteration) / cv
          gc.collect()
          
      df_test_site_meter = pd.DataFrame({"row_id": row_ids_site_meter, "meter_reading": y_pred_test_site_meter})
      df_test_sites_meters.append(df_test_site_meter)
      
      print("Scoring for site_id {}, meter {} completed \n".format(site, meter_id))
      gc.collect()










site_id:   0%|          | 0/16 [00:00<?, ?it/s]









meter_id:   0%|          | 0/4 [00:00<?, ?it/s]

Preparing test data for site_id 9
Scoring for site_id: 0, meter: 0


1103

7

7

...

7

7

Scoring for site_id 15, meter 3 completed 



0











meter_id: 100%|██████████| 4/4 [02:21<00:00, 35.55s/it]


















site_id: 100%|██████████| 16/16 [09:51<00:00, 61.85s/it]










In [138]:
df_test_sites_meters

[          row_id  meter_reading
 0              0       3.046064
 1            129       3.050258
 2            258       3.047829
 3            387       3.052265
 4            516       3.052265
 ...          ...            ...
 2260075  2259553       3.535546
 2260076  2259682       3.531625
 2260077  2259811       3.528897
 2260078  2259940       3.519317
 2260079  2260069       3.484238
 
 [1839600 rows x 2 columns]]

In [0]:
submit = pd.concat(df_test_sites_meters)
submit.meter_reading = np.clip(np.expm1(submit.meter_reading), 0, a_max=None)
submit.to_csv("submission.csv", index=False)

In [163]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 37615439
Data columns (total 2 columns):
row_id           int32
meter_reading    float64
dtypes: float64(1), int32(1)
memory usage: 795.3 MB


In [164]:
submit.head()

,row_id,meter_reading
0,0,20.032389
1,129,20.120798
2,258,20.069562
3,387,20.163220
4,516,20.163220
